In [3]:
import pandas as pd
import numpy as np
import joblib
import recipes as r

# Подготовка данных

In [4]:
def remove_columns():
    with open('./data/cols.txt', 'r') as file:
        columns_to_keep = file.readline().strip().split(',')

    df = pd.read_csv('./data/epi_r.csv')
    df = df[columns_to_keep]
    return df

In [5]:
df = remove_columns()
df.head()

,rating,almond,amaretto,anchovy,anise,apple,apple juice,apricot,artichoke,arugula,...,wheat/gluten-free,whiskey,white wine,whole wheat,wild rice,wine,yellow squash,yogurt,yuca,zucchini
0,2.500,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Подбор лучшей регрессии

# Linear Regression

In [6]:
model = r.Regressor()
model.train_test(df, 'rating1')
model.create_model(r.LinearRegression)
model.fit_model()
model.predict_model()
model.cross_validation()

train_test: error "['rating1'] not found in axis"
fit_model: error This LinearRegression estimator requires y to be passed, but the target y is None.
==========Predict Model==========
predict_model: error This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.
==========Cross-Validation==========
cross_validation: error The 'X' parameter of cross_val_score must be an array-like or a sparse matrix. Got None instead.


In [7]:
model_lr = r.Regressor()
model_lr.train_test(df, 'rating')
param_grid = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'positive': [True, False]
}
model_lr.create_model(r.LinearRegression)
model_lr.parameter_search(param_grid)
model_lr.predict_model()
model_lr.cross_validation()

==========Parameter Search==========
Best params: {'copy_X': True, 'fit_intercept': True, 'positive': False}
Best score: 1.6985
==========Predict Model==========
Train: MSE = 1.5729 | RMSE = 1.2542
Test: MSE = 1.5842 | RMSE = 1.2586
==========Cross-Validation==========
MSE = 1.712 | RMSE = 1.3085
MSE = 1.6769 | RMSE = 1.2949
MSE = 1.6543 | RMSE = 1.2862
MSE = 1.6617 | RMSE = 1.2891


In [8]:
# joblib.dump(model_lr, './model/model_lr.pkl')

In [9]:
# model_lr = joblib.load('./model/model_lr.pkl')

## Decision Tree Regressor

In [10]:
model = r.Regressor()
model.train_test(df, 'rating')
model.create_model(r.DecisionTreeRegressor, {'random_state': 21})
model.fit_model()
model.predict_model()
model.cross_validation()

==========Predict Model==========
Train: MSE = 0.22597 | RMSE = 0.47537
Test: MSE = 2.7472 | RMSE = 1.6575
==========Cross-Validation==========
MSE = 2.8865 | RMSE = 1.699
MSE = 2.9056 | RMSE = 1.7046
MSE = 2.7655 | RMSE = 1.663
MSE = 2.8076 | RMSE = 1.6756


In [11]:
model_dtr = r.Regressor()
model_dtr.train_test(df, 'rating')
param_grid = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    'splitter': ['best', 'random'],
    'max_depth': [None],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'min_weight_fraction_leaf': [0.0],
    'max_features': ['sqrt', 'log2']
}
model_dtr.create_model(r.DecisionTreeRegressor, {'random_state': 21})
model_dtr.parameter_search(param_grid)
model_dtr.predict_model()
model_dtr.cross_validation()

==========Parameter Search==========
Best params: {'criterion': 'poisson', 'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'splitter': 'random'}
Best score: 2.1431
==========Predict Model==========
Train: MSE = 1.0538 | RMSE = 1.0265
Test: MSE = 2.0443 | RMSE = 1.4298
==========Cross-Validation==========
MSE = 2.2545 | RMSE = 1.5015
MSE = 2.1597 | RMSE = 1.4696
MSE = 2.0829 | RMSE = 1.4432
MSE = 2.1099 | RMSE = 1.4525


In [12]:
model_dtr = r.Regressor()
model_dtr.train_test(df, 'rating')
param_grid = {
    'criterion': ['poisson'],
    'splitter': ['random'],
    'max_depth': np.arange(10, 100, 10),
    'min_samples_split': np.arange(2, 20, 1),
    'min_samples_leaf': np.arange(1, 20, 1),
    'min_weight_fraction_leaf': [0.0],
    'max_features': ['log2']
}
model_dtr.create_model(r.DecisionTreeRegressor, {'random_state': 21})
model_dtr.parameter_search(param_grid)
model_dtr.predict_model()
model_dtr.cross_validation()

==========Parameter Search==========
Best params: {'criterion': 'poisson', 'max_depth': np.int64(50), 'max_features': 'log2', 'min_samples_leaf': np.int64(7), 'min_samples_split': np.int64(17), 'min_weight_fraction_leaf': 0.0, 'splitter': 'random'}
Best score: 1.7166
==========Predict Model==========
Train: MSE = 1.6936 | RMSE = 1.3014
Test: MSE = 1.6774 | RMSE = 1.2952
==========Cross-Validation==========
MSE = 1.7573 | RMSE = 1.3256
MSE = 1.6808 | RMSE = 1.2965
MSE = 1.6725 | RMSE = 1.2933
MSE = 1.7257 | RMSE = 1.3137


In [13]:
# joblib.dump(model_dtr, './model/model_dtr.pkl')

In [14]:
# model_dtr = joblib.load('./model/model_dtr.pkl')

## Random Forest Regressor

In [15]:
model = r.Regressor()
model.train_test(df, 'rating')
model.create_model(r.RandomForestRegressor, {'random_state': 21})
model.fit_model()
model.predict_model()
model.cross_validation()

==========Predict Model==========
Train: MSE = 0.41766 | RMSE = 0.64626
Test: MSE = 1.722 | RMSE = 1.3123
==========Cross-Validation==========
MSE = 1.798 | RMSE = 1.3409
MSE = 1.7681 | RMSE = 1.3297
MSE = 1.7454 | RMSE = 1.3211
MSE = 1.7524 | RMSE = 1.3238


In [16]:
model_rfr = r.Regressor()
model_rfr.train_test(df, 'rating')
param_grid = {
    'n_estimators': [10],
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    'max_depth': [None],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'max_features': ['sqrt', 'log2']
}
model_rfr.create_model(r.RandomForestRegressor, {'random_state': 21})
model_rfr.parameter_search(param_grid)
model_rfr.predict_model()
model_rfr.cross_validation()

==========Parameter Search==========
Best params: {'criterion': 'poisson', 'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 10}
Best score: 1.6931
==========Predict Model==========
Train: MSE = 1.07 | RMSE = 1.0344
Test: MSE = 1.5883 | RMSE = 1.2603
==========Cross-Validation==========
MSE = 1.7181 | RMSE = 1.3108
MSE = 1.6386 | RMSE = 1.2801
MSE = 1.6434 | RMSE = 1.282
MSE = 1.6622 | RMSE = 1.2893


In [17]:
model_rfr = r.Regressor()
model_rfr.train_test(df, 'rating')
param_grid = {
    'n_estimators': [10],
    'criterion': ['poisson'],
    'max_depth': np.arange(10, 100, 10),
    'min_samples_split': np.arange(2, 20, 1),
    'min_samples_leaf': np.arange(1, 20, 1),
    'max_features': ['log2']
}
model_rfr.create_model(r.RandomForestRegressor, {'random_state': 21})
model_rfr.parameter_search(param_grid)
model_rfr.predict_model()
model_rfr.cross_validation()

==========Parameter Search==========
Best params: {'criterion': 'poisson', 'max_depth': np.int64(60), 'max_features': 'log2', 'min_samples_leaf': np.int64(1), 'min_samples_split': np.int64(18), 'n_estimators': 10}
Best score: 1.6544
==========Predict Model==========
Train: MSE = 1.4448 | RMSE = 1.202
Test: MSE = 1.5783 | RMSE = 1.2563
==========Cross-Validation==========
MSE = 1.68 | RMSE = 1.2961
MSE = 1.642 | RMSE = 1.2814
MSE = 1.6471 | RMSE = 1.2834
MSE = 1.6401 | RMSE = 1.2807


In [18]:
# joblib.dump(model_rfr, './model/model_rfr.pkl')

In [19]:
# model_rfr = joblib.load('./model/model_rfr.pkl')

## Bagging Regressor

In [20]:
model = r.Regressor()
model.train_test(df, 'rating')
model.create_model(r.BaggingRegressor, {'estimator': model_dtr.model, 'random_state': 21, 'n_jobs': -1})
model.fit_model()
model.predict_model()
model.cross_validation()

==========Predict Model==========
Train: MSE = 1.6361 | RMSE = 1.2791
Test: MSE = 1.5958 | RMSE = 1.2632
==========Cross-Validation==========
MSE = 1.7157 | RMSE = 1.3098
MSE = 1.6259 | RMSE = 1.2751
MSE = 1.6774 | RMSE = 1.2952
MSE = 1.6843 | RMSE = 1.2978


In [21]:
model_br = r.Regressor()
model_br.train_test(df, 'rating')
param_grid = {
    'n_estimators': [10],
    'max_samples': [1],
    'max_features': [1],
    'bootstrap': [True, False],
    'bootstrap_features': [True, False]
}
model_br.create_model(r.BaggingRegressor, {'estimator': model_dtr.model, 'random_state': 21, 'n_jobs': -1})
model_br.parameter_search(param_grid)
model_br.predict_model()
model_br.cross_validation()

==========Parameter Search==========
Best params: {'bootstrap': True, 'bootstrap_features': True, 'max_features': 1, 'max_samples': 1, 'n_estimators': 10}
Best score: 1.8673
==========Predict Model==========
Train: MSE = 2.0452 | RMSE = 1.4301
Test: MSE = 1.9266 | RMSE = 1.388
==========Cross-Validation==========
MSE = 3.5988 | RMSE = 1.8971
MSE = 1.9027 | RMSE = 1.3794
MSE = 1.8378 | RMSE = 1.3557
MSE = 2.0568 | RMSE = 1.4342


In [22]:
model_br = r.Regressor()
model_br.train_test(df, 'rating')
param_grid = {
    'n_estimators': np.arange(10, 100, 10),
    'max_samples': np.arange(1, 10, 1),
    'max_features': np.arange(1, 10, 1),
    'bootstrap': [True],
    'bootstrap_features': [True]
}
model_br.create_model(r.BaggingRegressor, {'estimator': model_dtr.model, 'random_state': 21, 'n_jobs': -1})
model_br.parameter_search(param_grid)
model_br.predict_model()
model_br.cross_validation()

==========Parameter Search==========
Best params: {'bootstrap': True, 'bootstrap_features': True, 'max_features': np.int64(4), 'max_samples': np.int64(9), 'n_estimators': np.int64(90)}
Best score: 1.8199
==========Predict Model==========
Train: MSE = 1.8215 | RMSE = 1.3496
Test: MSE = 1.7098 | RMSE = 1.3076
==========Cross-Validation==========
MSE = 1.8517 | RMSE = 1.3608
MSE = 1.7644 | RMSE = 1.3283
MSE = 1.8352 | RMSE = 1.3547
MSE = 1.8411 | RMSE = 1.3569


In [23]:
# joblib.dump(model_br, './model/model_br.pkl')

In [24]:
# model_br = joblib.load('./model/model_br.pkl')

## Voting Regressor

In [25]:
model_vr = r.Regressor()
model_vr.train_test(df, 'rating')
estimators = [
    ('dtr', model_dtr.model),
    ('lr', model_lr.model),
    ('rfr', model_rfr.model)
]
final_estimator = r.LinearRegression()
model_vr.create_model(r.VotingRegressor, {'estimators': estimators, 'n_jobs': -1})
model_vr.fit_model()
model_vr.predict_model()
model_vr.cross_validation()

==========Predict Model==========
Train: MSE = 1.5263 | RMSE = 1.2355
Test: MSE = 1.5722 | RMSE = 1.2539
==========Cross-Validation==========
MSE = 1.678 | RMSE = 1.2954
MSE = 1.6189 | RMSE = 1.2724
MSE = 1.6233 | RMSE = 1.2741
MSE = 1.6409 | RMSE = 1.281


In [26]:
model_vr = r.Regressor()
model_vr.train_test(df, 'rating')
param_grid = {
    'weights': [[1, 2, 3], [3, 2, 1], [2, 1, 3]]
}
model_vr.create_model(r.VotingRegressor, {'estimators': estimators, 'n_jobs': -1})
model_vr.parameter_search(param_grid)
model_vr.predict_model()
model_vr.cross_validation()

==========Parameter Search==========
Best params: {'weights': [1, 2, 3]}
Best score: 1.6401
==========Predict Model==========
Train: MSE = 1.4932 | RMSE = 1.2219
Test: MSE = 1.5629 | RMSE = 1.2502
==========Cross-Validation==========
MSE = 1.6703 | RMSE = 1.2924
MSE = 1.6199 | RMSE = 1.2728
MSE = 1.6229 | RMSE = 1.2739
MSE = 1.6313 | RMSE = 1.2772


In [27]:
# joblib.dump(model_vr, './model/model_vr.pkl')

In [28]:
# model_vr = joblib.load('./model/model_vr.pkl')

## Stacking Regressor

In [29]:
model_sr = r.Regressor()
model_sr.train_test(df, 'rating')
estimators = [
    ('dt', r.DecisionTreeRegressor(random_state=21)),
    ('lr', r.LinearRegression()),
    ('rf', r.RandomForestRegressor(n_estimators=10, random_state=21))
]
final_estimator = r.LinearRegression()
model_sr.create_model(r.StackingRegressor, {'estimators': estimators, 'final_estimator': final_estimator, 'cv': 2, 'n_jobs': -1})
model_sr.fit_model()
model_sr.predict_model()
model_sr.cross_validation()

==========Predict Model==========
Train: MSE = 1.0821 | RMSE = 1.0402
Test: MSE = 1.5265 | RMSE = 1.2355
==========Cross-Validation==========
MSE = 1.6477 | RMSE = 1.2836
MSE = 1.5882 | RMSE = 1.2602
MSE = 1.6005 | RMSE = 1.2651
MSE = 1.6136 | RMSE = 1.2703


In [30]:
model_sr = r.Regressor()
model_sr.train_test(df, 'rating')
estimators = [
    ('dtr', model_dtr.model),
    ('lr', model_lr.model),
    ('rfr', model_rfr.model)
]
final_estimator = r.LinearRegression()
model_sr.create_model(r.StackingRegressor, {'estimators': estimators, 'final_estimator': final_estimator, 'cv': 2, 'n_jobs': -1})
model_sr.fit_model()
model_sr.predict_model()
model_sr.cross_validation()

==========Predict Model==========
Train: MSE = 1.4545 | RMSE = 1.206
Test: MSE = 1.5611 | RMSE = 1.2494
==========Cross-Validation==========
MSE = 1.6694 | RMSE = 1.2921
MSE = 1.6237 | RMSE = 1.2742
MSE = 1.6256 | RMSE = 1.275
MSE = 1.6266 | RMSE = 1.2754


In [31]:
# joblib.dump(model_sr, './model/model_sr.pkl')

In [32]:
# model_sr = joblib.load('./model/model_sr.pkl')

# Предсказание

In [33]:
def func(lst):
    x = []

    for item in df.columns:
        if item in lst:
            x.append(1)
            lst.remove(item)
        else:
            x.append(0)

    if len(lst) == 0:
        x = pd.DataFrame([x], columns=df.columns)
        x.drop(['rating'], axis=1, inplace=True)
        return x
    else:
        raise Exception(f'{lst} нет таких ингредиентов')

# Подбор лучшего классификатора

## Бинаризация целевого столбца и разбиение на подвыборки

In [34]:
classing = r.Classification()
classing.binarization_round(df, 'rating')
classing.tr_tst_split(df, 'rating')
df

,rating,almond,amaretto,anchovy,anise,apple,apple juice,apricot,artichoke,arugula,...,wheat/gluten-free,whiskey,white wine,whole wheat,wild rice,wine,yellow squash,yogurt,yuca,zucchini
0,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20050,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Логистическая регрессия

In [35]:
classing.model_choose('LogisticRegression', {})
logreg_params = {
    'penalty': ['l2'],
    'C': [0.1, 1.0, 10.0],
    'solver': ['lbfgs', 'saga'],
    'random_state': [21]
}
best_logreg_params = classing.parameter_select(logreg_params)
best_logreg_params

Перекрестная проверка: 0.6683126265612458
Точность на тестовой выборке: 0.6788830715532286


{'C': 1.0, 'penalty': 'l2', 'random_state': 21, 'solver': 'saga'}

### Дерево решений

In [36]:
classing.model_choose('DecisionTreeClassifier', {})
tree_params = {
    "max_depth": [i for i in range(1, 50)],
    "class_weight": ['balanced', None],
    "criterion": ['entropy', 'gini'],
    'random_state': [21]
}
best_tree_params = classing.parameter_select(tree_params)
best_tree_params

Перекрестная проверка: 0.6649711796300821
Точность на тестовой выборке: 0.6761406133133882


{'class_weight': None, 'criterion': 'gini', 'max_depth': 8, 'random_state': 21}

### Случайный лес

In [37]:
classing.model_choose('RandomForestClassifier', {})
forest_params1 = {
    'n_estimators': [5, 10, 15, 25, 50, 100],
    'max_depth': [i for i in range(1, 50)],
    "class_weight": [None],
    "criterion": ['entropy'],
    'random_state': [21]
}
best_forest_params = classing.parameter_select(forest_params1)
best_forest_params

Перекрестная проверка: 0.6839218707158804
Точность на тестовой выборке: 0.6930939915233109


{'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 47,
 'n_estimators': 100,
 'random_state': 21}

## Расчет точности наивного классификатора

In [38]:
native_acc = classing.native_accuracy()
native_acc

0.6706556968337073

## Повторная бинаризация целевого столбца и разбиение на подвыборки

In [39]:
classing.binarization_category(df, 'rating')
df

,rating,almond,amaretto,anchovy,anise,apple,apple juice,apricot,artichoke,arugula,...,wheat/gluten-free,whiskey,white wine,whole wheat,wild rice,wine,yellow squash,yogurt,yuca,zucchini
0,so-so,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,great,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,great,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,great,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,so-so,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,so-so,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,great,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,great,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20050,great,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
classing.tr_tst_split(df, 'rating')

## Повторная классификация

### Логистическая регрессия

In [41]:
classing.model_choose('LogisticRegression', {})
logreg_params = {
    'penalty': ['l2'],
    'C': [0.1, 1.0, 10.0],
    'solver': ['lbfgs', 'saga'],
    'random_state': [21]
}
best_logreg_params = classing.parameter_select(logreg_params)
best_logreg_params

Перекрестная проверка: 0.8001696456937933
Точность на тестовой выборке: 0.8022936923460484


{'C': 1.0, 'penalty': 'l2', 'random_state': 21, 'solver': 'saga'}

### Дерево решений

In [42]:
classing.model_choose('DecisionTreeClassifier', {})
tree_params = {
    "max_depth": [i for i in range(1, 50)],
    "class_weight": ['balanced', None],
    "criterion": ['entropy', 'gini'],
    'random_state': [21]
}
best_tree_params = classing.parameter_select(tree_params)
best_tree_params

Перекрестная проверка: 0.7983243710718219
Точность на тестовой выборке: 0.7973074046372476


{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'random_state': 21}

### Случайный лес

In [43]:
classing.model_choose('RandomForestClassifier', {})
forest_params1 = {
    'n_estimators': [5, 10, 15, 25, 50, 100],
    'max_depth': [i for i in range(1, 50)],
    "class_weight": [None],
    "criterion": ['entropy'],
    'random_state': [21]
}
best_forest_params = classing.parameter_select(forest_params1)
best_forest_params

Перекрестная проверка: 0.8059546595988214
Точность на тестовой выборке: 0.8087758663674894


{'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 49,
 'n_estimators': 100,
 'random_state': 21}

## Повторный расчет точности наивного классификатора

In [44]:
native_acc = classing.native_accuracy()
native_acc

0.7978060334081276

## Что хуже: предсказать плохой рейтинг, который хорош в реальной жизни, или предсказать хороший рейтинг, который плох в реальной жизни?

### Ответ: предсказать хороший рейтинг, который плох в реальной жизни

## Повторная классификация с ансамблями

### Инициализация моделей

In [45]:
best_logreg_params = {'C': 1.0, 'penalty': 'l2', 'random_state': 21, 'solver': 'saga'}
best_tree_params = {'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'random_state': 21}
best_forest_params = {'class_weight': None, 'criterion': 'entropy', 'max_depth': 49, 'n_estimators': 100, 'random_state': 21}

In [46]:
logreg = classing.model_choose('LogisticRegression', best_logreg_params)
tree = classing.model_choose('DecisionTreeClassifier', best_tree_params)
forest = classing.model_choose('RandomForestClassifier', best_forest_params)

### VotingClassifier

In [47]:
voting_cls = classing.model_choose('VotingClassifier', {'estimators': [
        ('logreg', logreg),
        ('tree', tree),
        ('forest', forest)
    ]})
voting_grid = {
    'voting': ['hard', 'soft'],
    'weights': [
        None,
        [1, 2, 1],
        [2, 1, 1],
        [1, 1, 2]
    ]}
best_voting_params = classing.parameter_select(voting_grid)

Перекрестная проверка: 0.8029125142765127
Точность на тестовой выборке: 0.806033408127649


### StackingClassifier

In [48]:
base_models = [
    ('logreg', logreg),
    ('tree', tree),
    ('forest', forest)
]
stack_cls = classing.model_choose('StackingClassifier', {'estimators': base_models})
stack_grid = {
    'final_estimator': [logreg, tree, forest],
    'n_jobs': [-1],
}
best_stack_params = classing.parameter_select(stack_grid)

Перекрестная проверка: 0.8041093974114826
Точность на тестовой выборке: 0.8077786088257293


## Итоговая модель (с лучшей точностью)

In [49]:
best_model = classing.model_choose('RandomForestClassifier', best_forest_params)
classing.fit()


## Сохранение модели

In [50]:
joblib.dump(best_model, 'main_model.pkl')

['main_model.pkl']